In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numba 
from molsim import (
    mullerBrownPotential,
    mullerBrownPotentialAndGradient, 
    plot_muller_brown_heatmap
)

# Parallel Tempering

In parallel tempering we consider N systems. In each of these systems we perform a simula-
tion in the canonical ensemble, but each system is in a different thermodynamic state. Usually, but
not necessarily, these states differ in temperature. In what follows we assume that this is the case.
Systems with a sufficiently high temperature pass all barriers in the system. The low-temperature
systems, on the other hand, mainly probe the local energy minima. The idea of parallel tempering
is to include MC trial moves that attempt to “swap” systems that belong to different thermo-
dynamic states, e.g., to swap a high temperature system with a low temperature system. If the
temperature difference between the two systems is very large, such a swap has a very low prob-
ability of being accepted. This is very similar to particle displacement in ordinary Monte Carlo.
If one uses a very large maximum displacement a move has a very low probability of being ac-
cepted. The solution to this problem is to use many small steps. In parallel tempering we use
intermediate temperatures in a similar way. Instead of making attempts to swap between a low
and a high temperature, we swap between systems with a small temperature difference. In prin-
ciple the distribution of the position of a particle should be symmetrical. The high-temperature
system does show this symmetrical distribution.
The total partition function of a system with N canonical subsystems (Q) equals

$ Q = \Pi_{i=1}^{i=N} Q_i$

in which Qi is the canonical partition function of the individual system i

$ Q_i = \sum_{x_i} \exp [-\beta_i U(x_i)] $

where $\beta_i = 1/ (k_B T_i)$. For each of these systems, individual trial moves are performed. After a
randomly selected number of trial moves, an attempt is made to exchange configurations. Two
systems (i and j, $|i − j| = 1$) are selected at random, the systems are exchanged by choosing
$x_i (n) = x_j (o)$ and $x_j (n) = x_i (o)$. The ratio of acceptance probabilities equals

$\frac{acc(o \to n)}{acc(n \to o)} = \exp[(\beta_i - \beta_j) \times (U(x_i) - U(x_j))]$

Such trial moves will be accepted when there is enough overlap between the energies of systems
i and j. To demonstrate this technique, consider a two-dimensional system of a particle in a Muller-Brown potential. While a particle may have the thermal energy to visit another local minimum than the global minimum, it may not have the energy to overcome the barrier. Coupling this system to a system at a higher temperature that is able to sample the barrier region allows the system to visit all states that it should be able to reach.

## Question 1
Derive the acceptance criterium:
$\frac{acc(o \to n)}{acc(n \to o)} = \exp[(\beta_i - \beta_j) \times (U(x_i) - U(x_j))]$

from the detailed balance equation
$acc(o \to n) = \alpha(n \to o) \mathcal{N}(n)$

## Question 2
Now that you have derived the detailed balance equation, try to understand the following Monte Carlo algorithm. What two moves happen each cycle? What parameters can we set? How do we specify whether we do parallel tempering?

If you look at the acceptance criterium, we currently never accept 

In [ ]:
@numba.njit
def monteCarlo(temperatures: np.ndarray, 
               numberOfCycles: int, 
               parallelTemperingProbability: float = 0.0, 
               maxDisplacement: float = 0.1) -> np.ndarray:
    """
    Perform a Monte Carlo simulation of the Muller-Brown potential landscape with optional parallel tempering.

    Parameters
    ----------
    temperatures : np.ndarray
        An array of temperature values for each system. Shape: (numberOfSystems,).
    numberOfCycles : int
        The number of Monte Carlo cycles to perform.
    parallelTemperingProbability : float, optional
        The probability of attempting a parallel tempering swap at each cycle. Default is 0.0 (no parallel tempering).
    maxDisplacement : float, optional
        The maximum displacement scale for proposing a new position move. Default is 0.1.

    Returns
    -------
    positions : np.ndarray
        The array of positions for all systems across all cycles. 
        Shape: (numberOfCycles, numberOfSystems, 2).
    
    Notes
    -----
    - Each system is simulated at a given temperature (from the input `temperatures`).
    - The function uses the Muller-Brown potential as the energy function.
    - Parallel tempering moves are attempted with a certain probability, which may help the simulation escape local minima.

    """
    numberOfSystems: int = len(temperatures)

    # Initialize arrays to store positions and energies
    # Using double precision floats for positions and energies
    positions = np.zeros((numberOfCycles, numberOfSystems, 2), dtype=np.float64)
    energies = np.zeros((numberOfCycles, numberOfSystems), dtype=np.float64)

    # Compute inverse temperatures (betas)
    # beta = 1 / T
    betas = [1.0 / T for T in temperatures]

    # Compute displacement scales based on temperatures, so that moves scale with sqrt(T)
    displacements = [np.sqrt(T) * maxDisplacement for T in temperatures]

    # Track acceptance rates for normal moves and parallel tempering moves (attempted, accepted) pairs per system
    translationAcceptance = np.zeros((numberOfSystems, 2), dtype=np.float64)
    parallelTemperingAcceptance = np.zeros((numberOfSystems - 1, 2), dtype=np.float64)

    # The known minimum position on the Muller-Brown surface to start from
    minimum = np.array([-0.557114228, 1.44889779], dtype=np.float64)

    # Initialize the first cycle positions at the known minimum
    # Repeat the minimum position for all systems
    positions[0] = np.repeat(minimum, numberOfSystems).reshape(2, numberOfSystems).T

    # Compute initial energies for all systems
    for system in range(numberOfSystems):
        # mullerBrownPotential is assumed defined elsewhere
        energies[0, system] = mullerBrownPotential(positions[0, system, 0], positions[0, system, 1])

    # Main Monte Carlo loop
    for cycle in range(1, numberOfCycles):
        # Attempt position updates for each system
        for system in range(numberOfSystems):
            # Propose a new position by random displacement
            translationAcceptance[system, 0] += 1
            newPosition = positions[cycle - 1, system] + displacements[system] * (np.random.rand(2) - 0.5)
            newEnergy = mullerBrownPotential(newPosition[0], newPosition[1])

            # Metropolis acceptance criterion
            if np.random.rand() < np.exp(-betas[system] * (newEnergy - energies[cycle - 1, system])):
                # Accept the move
                positions[cycle, system] = newPosition
                energies[cycle, system] = newEnergy
                translationAcceptance[system, 1] += 1
            else:
                # Reject the move, keep old position and energy
                positions[cycle, system] = positions[cycle - 1, system]
                energies[cycle, system] = energies[cycle - 1, system]

        # Attempt parallel tempering swap with a given probability
        if np.random.rand() < parallelTemperingProbability:
            systemA = np.random.randint(0, numberOfSystems - 1)
            systemB = systemA + 1

            parallelTemperingAcceptance[systemA, 0] += 1  # Count the swap attempt
            # Acceptance criterion for parallel tempering
            if np.random.rand() < np.exp((betas[systemB] - betas[systemA]) * (energies[cycle, systemB] - energies[cycle, systemA])):
                parallelTemperingAcceptance[systemA, 1] += 1  # Count the successful swap

                # Swap positions
                tmp_pos = positions[cycle, systemB].copy()
                positions[cycle, systemB] = positions[cycle, systemA]
                positions[cycle, systemA] = tmp_pos

                # Swap energies
                tmp_en = energies[cycle, systemB]
                energies[cycle, systemB] = energies[cycle, systemA]
                energies[cycle, systemA] = tmp_en

    # Print acceptance rates for debugging or analysis
    print("Translation acceptance rates:", translationAcceptance[:, 1] / translationAcceptance[:, 0])
    if parallelTemperingProbability > 0.0:
        print("Parallel tempering acceptance rates:", parallelTemperingAcceptance[:, 1] / parallelTemperingAcceptance[:, 0])

    return positions


In [ ]:
temperatures = [0.5, 3.0, 5.0, 11.0]
numberOfCycles = int(2e7)

In [ ]:
positions = monteCarlo(temperatures, numberOfCycles, parallelTemperingProbability=0.0)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
ax = ax.flatten()
for i in range(4):
    plot_muller_brown_heatmap(ax[i])
    ax[i].scatter(*positions[:, i].T, s=0.5, c='red', label=f"T={temperatures[i]}")
    ax[i].set_title(f"T={temperatures[i]}", c='red')
fig.tight_layout()

In [ ]:
ptPositions = monteCarlo(temperatures, numberOfCycles, parallelTemperingProbability=0.5)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
ax = ax.flatten()
for i in range(4):
    plot_muller_brown_heatmap(ax[i])
    ax[i].scatter(*ptPositions[:, i].T, s=0.5, c='red', label=f"T={temperatures[i]}")
    ax[i].set_title(f"T={temperatures[i]}", c='red')
fig.tight_layout()